In [ ]:
from jax import numpy as jnp
from jax import vmap
from jax.config import config

from qutip import basis, tensor, qeye, sigmax, sigmay, sigmaz, Qobj, fidelity
from qutip.superop_reps import to_kraus, to_chi, to_choi, sprepost, kraus_to_choi, choi_to_chi, _pauli_basis, choi_to_kraus, kraus_to_choi
from qutip.qip.operations import rx, ry, rz
from qutip.tomography import qpt_plot_combined
from qutip.tomography import qpt as qpt_matrix

import numpy as np
import qutip as qt
from plotting_functions import hinton_phase, plot_comparison
from qpt_utils import tensor_product_list, dag, convert_to_jax, plot_comparison, Choi
from qpt_functions import *

from tqdm.auto import tqdm

import matplotlib.pyplot as plt
from matplotlib import cm
from qutip import settings

from itertools import product

config.update('jax_enable_x64', True)

import scipy.optimize as opt

#plt.rc('text', usetex=True)
# plt.rc('text.latex', preamble=r'\usepackage{amsmath} \usepackage{physics}')

%load_ext autoreload
%autoreload 2

# Load the data

In [ ]:
n_qubit = 3

N_samples = 10000 # Number of samples for each probe and measurement going into the outcome frequencies

phi=np.linspace(0, -np.pi, 5)  # Some sign issue I wasn't able to trace back might need to change here to get the proper phases from the data
phi_id = 0 # Choose which phi to reconstruct out of [0, pi/4, pi/2, 3pi/2, pi]

targ_phi = phi[phi_id]

# Shape = [phi_id, n_probes, n_meas, outcome_freqs] = [5, 4^n_qubit, 3^n_qubit, 2^N_qubits]
data = np.load(r'..\..\Data\Fig3 Data\QPT_Data\3Q_QPT_250ns_all_phi.npy')


# Select just the data corresponding to phi target
data_raw = data[phi_id, :, :, :]

# Constructing the ideal initial states, rotation operators and measurement POVMs

In [ ]:
op_basis = [[qt.qeye(2), qt.sigmax(), qt.sigmay(), qt.sigmaz()]] * n_qubit
op_label = [['i', 'x', 'y', 'z']] * n_qubit

# Generate measurement labels
measurement_labels = list(product(['0', '1'], repeat=n_qubit))

#Note the sequence for the measurements are slightly different for the processing
#Introduce measurement mask to switch order of each state probability in the array
# mapping from |Q2,Q1,Q0> to |Q0,Q1,Q2>
measurement_idx = (0, 4, 2, 6, 1, 5, 3, 7)
measurement_labels = ["".join(p) for p in measurement_labels]

#Basis probe states
ground = basis(2, 0)
excited = basis(2, 1)
plus = (ground + excited).unit()
imag = (ground + 1j*excited).unit()

probes = tensor_product_list([ground*ground.dag(), excited*excited.dag(), plus*plus.dag(), imag*imag.dag()], repeat=n_qubit)
probe_labels = list(product(['0', '1', '+', "i"], repeat=n_qubit))

#Basis measurement operators
pauli_basis_labels = list(product(['I', 'X', 'Y', 'Z'], repeat=n_qubit))
pauli_basis = tensor_product_list([qeye(2), sigmax(), sigmay(), sigmaz()], repeat=n_qubit)

# Single qubit rotation operators
# X -> ry(-pi/2) single qubit rotation about y, Y -> Rotx(pi/2), Z -> Identity
# list(product(['X', 'Y', 'Z'], repeat=3))
X, Y, Z = ry(-jnp.pi/2), rx(jnp.pi/2), qeye(2)
rotations = tensor_product_list([X, Y, Z], repeat=n_qubit)
rotation_labels = list(product(['X', 'Y', 'Z'], repeat=n_qubit))

# Constructing SPAM Characterized Operators, Initial states and POVMs

In [ ]:
# Constructing the operators reported from the GST experiments

# Noisy rotation generators for preparations
# Angles for Ry(pi), Ry(pi/2), Rx(-pi/2) 
q0_angs = [1.00029, 0.510156, 0.509419]
q0_axis = [[-0.0065302, 0.9999786, 0.0003137], 
           [0.110777, 0.9999384, -0.0006218], 
           [-0.9999167, 0.0129077, 6.2e-6]]

q1_angs = [1.000024, 0.514077, 0.514256]
q1_axis = [[-0.0051505, 0.9999867, -0.0001557], 
           [0.0118059, 0.9999303, 0.0003109], 
           [-0.9999362, 0.112991, 2.4e-6]]

q2_angs = [0.999292, 0.506918, 0.505728]
q2_axis = [[-0.0037203, 0.9999908, 0.0021185], 
           [0.0040367, 0.999983, -0.0042045], 
           [-0.9999944, 0.0033346, -3.8e-6]]

# Return noisy rotation operators for each qubit
ry_p0, ry_2p0, rx_2m0 = return_rotations(q0_angs, q0_axis)
ry_p1, ry_2p1, rx_2m1 = return_rotations(q1_angs, q1_axis)
ry_p2, ry_2p2, rx_2m2 = return_rotations(q2_angs, q2_axis)


rot_noisy = [list(return_rotations(q0_angs, q0_axis)), list(return_rotations(q1_angs, q1_axis)), list(return_rotations(q2_angs, q2_axis))]

# Construct Noisy measurement rotations for tomographyic pulse, the inverse of the pulse is just a 180deg phase shift which implements a perfect virtual-Z gate
X0 = ry_2p0.dag()
X1 = ry_2p1.dag()
X2 = ry_2p2.dag()

Y0 = rx_2m0.dag()
Y1 = rx_2m1.dag()
Y2 = rx_2m2.dag()

Z0 = qeye(2)
Z1 = qeye(2)
Z2 = qeye(2)

# Corrected rotation operators for each qubit
noisy_rotation_operators = {0:{'X':X0, 'Y':Y0, 'Z':Z0},
                                1:{'X':X1, 'Y':Y1, 'Z':Z1},
                                2:{'X':X2, 'Y':Y2, 'Z':Z2}}

# The tensor product for all rotation operators
noisy_rotations = []

for p in product(['X', 'Y', 'Z'], repeat=n_qubit):
    noisy_rotations.append(tensor(noisy_rotation_operators[0][p[0]],
                                  noisy_rotation_operators[1][p[1]],
                                  noisy_rotation_operators[2][p[2]]))

### State Initialization errors

In [ ]:
#Extracted ground states from single-qubit GST

# Removing small off-diagonal elements
# g0 = Qobj(np.array([[0.98267, 0.],
#                     [0.0, 0.01733]]))

# g1 = Qobj(np.array([[0.9786449, 0],
#                     [0,0.0213551]]))

# g2 = Qobj(np.array([[0.9831357, 0],
#                     [0, 0.0168643]]))


# Keeping all terms
g0 = Qobj(np.array([[0.98267, -0.0011495],
                    [0.0007066, 0.01733]]))

g1 = Qobj(np.array([[0.9786449, -0.0001291],
                    [0.0008762,0.0213551]]))

g2 = Qobj(np.array([[0.9831357, -0.0025322],
                    [-0.0009186, 0.0168643]]))

rhos_in = [g0, g1, g2]
#Generate all probe states by applying the noisy rotations to generate the noisy states [0, 1, +, +i]^3
noisy_probes = gen_new_probes(rhos_in, rot_noisy)

### Correction of measurement by implementing characterized POVMs from GST

In [ ]:
# Removed Small off-diagonal
# pi0 = {0:Qobj(np.array([[0.9979474, 0],
#                         [0, 0.0293532]]))}
# pi0[1] = qt.qeye(2) - pi0[0]


# pi1 = {0:Qobj(np.array([[0.9982515, 0],
#                         [0, 0.0273934]]))}
# pi1[1] = qt.qeye(2) - pi1[0]

# pi2 = {0:Qobj(np.array([[0.9959302, 0],
#                         [0, 0.0967759]]))}
# pi2[1] = qt.qeye(2) - pi2[0]




# Full data
pi0 = {0:Qobj(np.array([[0.9979474, -0.0010535],
                        [0.0012261, 0.0293532]]))}
pi0[1] = qt.qeye(2) - pi0[0]


pi1 = {0:Qobj(np.array([[0.9982515, -0.0001658],
                        [0.0023416, 0.0273934]]))}
pi1[1] = qt.qeye(2) - pi1[0]

pi2 = {0:Qobj(np.array([[0.9959302, -0.0021059],
                        [-0.0013668, 0.0967759]]))}
pi2[1] = qt.qeye(2) - pi2[0]

# Construct tensor of all measurement projectors
noisy_povms = []
for i in measurement_idx:
    idx = measurement_labels[i]
    ind1, ind2, ind3 = int(idx[0]), int(idx[1]), int(idx[2])
    noisy_povms.append(tensor(pi0[ind1], pi1[ind2], pi2[ind3]))

### Vectorize everything to jax for speedy implementation

In [ ]:
dag_vectorized = vmap(dag)

probes = convert_to_jax(probes)
noisy_probes = convert_to_jax(noisy_probes)

rotations = convert_to_jax(rotations)
noisy_rotations = convert_to_jax(noisy_rotations)

noisy_povms = convert_to_jax(noisy_povms)

pm = convert_to_jax(pauli_basis)
pn = dag_vectorized(pm)

In [ ]:
U_ideal = U_CCZS(np.pi/2, targ_phi, 0)

# Convert to superoperator
Urho_ideal = qt.spre(U_ideal)*qt.spost(U_ideal.dag())

# Dimension conversion to make U_ideal work with tensored qubit states (Convert from [8,8]-> [[2,2,2],[2,2,2]])
U_ideal.dims = [[2, 2, 2], [2, 2, 2]]

# Use qutip for some advance methods later
U_ideal_kraus = to_kraus(U_ideal)
U_ideal_choi = to_choi(U_ideal)
U_ideal_chi = to_chi(U_ideal_choi)

# Separate instance of same ideal but as arrays to save object conversion later
kraus_ideal_array = jnp.array([U_ideal.full()])
choi_ideal_array = Choi(kraus_ideal_array)




# Simulate the experimental data with the ideal probes, unitary, and measurements
output_states_ideal = apply_process(kraus_ideal_array, probes)
final_states_ideal = apply_rotation(output_states_ideal, rotations)
data_ideal = jnp.diagonal(final_states_ideal, axis1=2, axis2=3)

# Run a simulation using the ideal process with noisy probse, measurements, rotations to generate a null hypothesis data set assuming the ideal CCZS is the "true" process
output_states_null = apply_process(kraus_ideal_array, noisy_probes)
final_states_null = apply_rotation(output_states_null, noisy_rotations)
projected_outcomes_null = jnp.einsum('bnij, pkj -> bnpik', final_states_null, noisy_povms)
data_null = jnp.trace(projected_outcomes_null, axis1=3, axis2=4)


### Generate Initial guess by inverting vectorized process matrix and output data

In [ ]:
A0, A = gen_Amat(noisy_probes, noisy_rotations, noisy_povms)

rho0 = estimate_process(data_raw, A0, rho_true=choi_ideal_array)

### Plot a comparison of the ideal choi matrix and the initial guess

In [ ]:
plot_comparison(choi_ideal_array, rho0)

### Perform the PLS reconstruction algorithm

In [ ]:
rho_list, least_ev_pls = run_reconstruction(rho0, choi_ideal_array)

In [ ]:
# Plot the cost function as a function of the number of iterations
plt.plot(np.log10(-np.array(least_ev_pls)))
plt.tight_layout()


### Compare Ideal Unitary to the leading order reconstructed Kraus operator

In [ ]:
choi_reconstructed = rho_list[-1][0]*8 # renormalize to dimension
choi_reconstructed_qutip = Qobj(choi_reconstructed, dims = [[[2, 2, 2], [2, 2, 2]], [[2, 2, 2], [2, 2, 2]]], superrep='choi')

# Save the reconstruction for easy plotting and data processing without needing to rerun
# np.save('choi_reconstructed_Phi{}_250ns.npy'.format(phi_id), choi_reconstructed)

kraus_reconstructed_qutip = choi_to_kraus(choi_reconstructed_qutip)
chi_reconstructed_qutip = to_chi(choi_reconstructed_qutip)

hinton_phase(U_ideal_kraus[0], cmap=cm.get_cmap('viridis'))
hinton_phase(kraus_reconstructed_qutip[0], cmap=cm.get_cmap('viridis'))

### Compare Choi Matrices after running PLS reconstruction

In [ ]:
# Unit the matrices to deal with dimensionality definitions
fid = np.round(100*fidelity(chi_reconstructed_qutip.unit(), U_ideal_chi)**2)

plot_comparison(choi_ideal_array, choi_reconstructed, title=r"$\mathcal{}(U_{})$= {}".format(r'{F}', r'{CCZS}', fid))

# Find the angles that best match the data

In [ ]:
# Residual to minimize

def residual(x, data):
    # CCZS parameters
    theta = x[0]
    phi = x[1]
    gamma = x[2]
    # Local single qubit phase rotations (virtual-Z gates)
    z1 = x[3]
    z2 = x[4]
    z3 = x[5]

    Z_rot = tensor([rz(z1*np.pi), rz(z2*np.pi), rz(z3*np.pi)])

    U_trial = U_CCZS(theta*np.pi, phi*np.pi, gamma*np.pi)
    U_trial.dims = [[2, 2, 2], [2, 2, 2]]
    
    F = fidelity(to_chi(kraus_to_choi([data])).unit(), to_chi(kraus_to_choi([Z_rot*U_trial])).unit())**2
    # Minimize the infidelity
    return (1 - F)

In [ ]:
methods = ['Nelder-Mead', 'Powell', 'COBYLA']

res = opt.minimize(residual, [0.5, phi_id/4.0, 0, 0, 0, 0],
                        args=choi_reconstructed_qutip,
                        method=methods[0], tol=1e-16)
print(r'Angles (theta, phi, gamma, z1, z2, z3): ', res.x)
print(r'Process Fidelity: ', 1-res.fun)

# Perform bootstrapping

In [ ]:
from IPython import display
live_plot = True


bounds = ((-2,2),(-2,2),(-2,2),(-0.05,0.05),(-0.05,0.05),(-0.05,0.05))
n_boot = 100  # increase the number of bootstrapping samples for more representative statistics (Note: takes a while to run)
n_shots = 10000 # Number of resampling of data points NEEDS to match what was actually measured in experiment


F = []
F_opt = []
angles = []

for i in range(n_boot):
    # Resample the empirical distribution of the raw data from the process tomography experiment
    data_new = resample_data(data_raw, n=n_shots)
    # Perform an estimate of the initial choi matrix as we did previously
    rho_new = estimate_process(data_new, A0, rho_true=choi_ideal_array)
    #run the reconstruction on this newly sampled data
    rho_list_new, least_ev_pls_new = run_reconstruction(rho_new, choi_ideal_array)

    choi_reconstructed_new = rho_list_new[-1][0]*8
    choi_reconstructed_new_qutip = Qobj(choi_reconstructed_new, dims = [[[2, 2, 2], [2, 2, 2]], [[2, 2, 2], [2, 2, 2]]], superrep='choi')

    F.append(fidelity(to_chi(choi_reconstructed_new_qutip).unit(), U_ideal_chi)**2)

    res_new = opt.minimize(residual,
                           [0.5, phi_id/4, 0, 0, 0, 0], # there is sometimes a minus sign that is needed on the phi_id haven't been able to trace it back 
                           args=choi_reconstructed_new_qutip,
                           bounds=bounds,
                           method='Powell', tol=1e-16)

    angles.append(res_new.x)
    F_opt.append(1-res_new.fun)

    # For live plotting of the results
    if live_plot:
        plt.clf()
        plt.hist(F, bins='auto', alpha=0.75)
        plt.hist(F_opt, bins='auto', alpha=0.75)
        plt.title('Sample i={}, last point={:.3f}'.format(i+1, F[-1]*100))
        display.display(plt.gcf())
        display.clear_output(wait=True) 

# saving the results for further plotting/post-processing
save = False
if save:
    #Save unoptimized fidelity distribution
    np.save(r'BootStrapFidelity_Phi{}_250ns_6param.npy'.format(phi_id), F)
    #Save the control error free fidelity
    np.save(r'BootStrapOptFidelity_Phi{}_250ns_6param.npy'.format(phi_id), F_opt)
    #Save the CCZS angles
    np.save(r'BootStrapAngles_Phi{}_250ns_6param.npy'.format(phi_id), angles)
    


